In [4]:
## import statements ##
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms
% matplotlib inline

# New section

# New section

In [5]:
train_data = []

Reading the train data and storing them in a single dataframe

In [6]:
data_files = ['Youtube01-Psy.csv','Youtube02-KatyPerry.csv','Youtube03-LMFAO.csv','Youtube04-Eminem.csv','Youtube05-Shakira.csv']
for file in data_files:
    data = pd.read_csv(file)
    train_data.append(data)
train_data = pd.concat(train_data)

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 369
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  1956 non-null   object
 1   AUTHOR      1956 non-null   object
 2   DATE        1711 non-null   object
 3   CONTENT     1956 non-null   object
 4   CLASS       1956 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 91.7+ KB


In [8]:
train_data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [9]:
train_data['CLASS'].value_counts()

1    1005
0     951
Name: CLASS, dtype: int64

#### Data Cleaning

We don't need all the features. Hence drop all the columns except CONTENT, CLASS

In [10]:
## Function which drops the given features from the given dataframe
def drop_fectures(features,data):
    data.drop(features,axis=1,inplace=True)

In [11]:
drop_fectures(['COMMENT_ID','AUTHOR','DATE'],train_data)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 369
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CONTENT  1956 non-null   object
 1   CLASS    1956 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.8+ KB


In [13]:
import re
re.findall("[A-Za-z]+","Hey Ravi57,How are you doing?")

['Hey', 'Ravi', 'How', 'are', 'you', 'doing']

Processing the comments in such a way that they contains only alphabets.

In [14]:
def process_content(content):
    return " ".join(re.findall("[A-Za-z]+",content.lower()))

In [15]:
train_data['processed_content'] = train_data['CONTENT'].apply(process_content)

In [16]:
train_data.head()

,CONTENT,CLASS,processed_content
0,"Huh, anyway check out this you[tube] channel: ...",1,huh anyway check out this you tube channel kob...
1,Hey guys check out my new channel and our firs...,1,hey guys check out my new channel and our firs...
2,just for test I have to say murdev.com,1,just for test i have to say murdev com
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,me shaking my sexy ass on my channel enjoy
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1,watch v vtarggvgtwq check this out


Obeserve the processed_content column. The comments are converted to lower case and all the punctuations, symbols, numbers are removed.

In [17]:
drop_fectures(['CONTENT'],train_data)

Splitting the whole data into train and test sets

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data['processed_content'],train_data['CLASS'],test_size=0.33,random_state=57)

Using CountVectorizer for text preprocessing, tokenizing and filtering of stopwords. It builds a dictionary of features and transform documents to feature vectors.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
count_vect = CountVectorizer(stop_words='english')
x_train_counts = count_vect.fit_transform(x_train)

In [21]:
x_train_counts.shape

(1310, 2921)

##### From occurrences to frequencies
Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

Both **tf** and **tf–idf** can be computed as follows:

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tranformer = TfidfTransformer()
x_train_tfidf = tranformer.fit_transform(x_train_counts)
x_train_tfidf.shape

(1310, 2921)

In [23]:
x_test_counts = count_vect.transform(x_test)
x_test_counts

<646x2921 sparse matrix of type '<class 'numpy.int64'>'
	with 4004 stored elements in Compressed Sparse Row format>

In [24]:
x_test_tfidf = tranformer.transform(x_test_counts)
x_test_tfidf

<646x2921 sparse matrix of type '<class 'numpy.float64'>'
	with 4004 stored elements in Compressed Sparse Row format>

##### Model Selection

Let's use LogiticRegression for prediciting the 'CLASS' as it is most suitable for binary classification

In [25]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train_tfidf,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
predictions = model.predict(x_test_tfidf)

Hey! we got out predictions. Now Let's check the accuracy of our model.

In [27]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [28]:
confusion_matrix(y_test,predictions)

array([[300,  12],
       [ 31, 303]])

The above confusion matrix states that we classified (176 + 187) = 363 out of 392 comments correctly. Moreover, we only classify 5 genuine comments as spam when it was a real comment.

In [29]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93       312
           1       0.96      0.91      0.93       334

    accuracy                           0.93       646
   macro avg       0.93      0.93      0.93       646
weighted avg       0.94      0.93      0.93       646



naive bayes


In [30]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_tfidf,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
predictions = model.predict(x_test_tfidf)

In [32]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [33]:
confusion_matrix(y_test,predictions)

array([[267,  45],
       [ 26, 308]])

In [34]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88       312
           1       0.87      0.92      0.90       334

    accuracy                           0.89       646
   macro avg       0.89      0.89      0.89       646
weighted avg       0.89      0.89      0.89       646



random forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
predictions = model.predict(x_test_tfidf)

In [37]:
confusion_matrix(y_test,predictions)

array([[304,   8],
       [ 29, 305]])

In [38]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       312
           1       0.97      0.91      0.94       334

    accuracy                           0.94       646
   macro avg       0.94      0.94      0.94       646
weighted avg       0.94      0.94      0.94       646



svm

In [39]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler


In [40]:

model = SVC()
model.fit(x_train_tfidf,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [41]:
predictions = model.predict(x_test_tfidf)
confusion_matrix(y_test,predictions)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       312
           1       0.97      0.91      0.94       334

    accuracy                           0.94       646
   macro avg       0.94      0.94      0.94       646
weighted avg       0.94      0.94      0.94       646

